Pour une lecture améliorée, ce notebook est idéalement lu avec une extension qui gère les niveaux des titres Markdown.  
Par exemple avec l'extension 'Table of Contents' du module 'Nbextensions' de Jupyter.

# Mise en place du notebook

## Import des modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress as reg_lin
from mesfonctions import scatter_plot
from mesfonctions import arrondi
from mesfonctions import display_scores

from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.compose import TransformedTargetRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor

from graphviz import Source
from sklearn.tree import export_graphviz


In [2]:
# pour développement only - cellule à supprimer du notebook final
import mesfonctions
from importlib import reload  
mesfonctions = reload(mesfonctions)
# from mesfonctions import xxx

## Gestion des paramètres d'affichage de Jupyter

In [3]:
# pour gérer l'affichage des dataframes pandas
pd.options.display.max_rows = 100
pd.options.display.max_columns = 200

# pour gérer l'affichage des arrays numpy
# 230 sur ma config bureau, 160 sur ma config portable
np.set_printoptions(linewidth=160)

# pour afficher les gros fichiers sur toute la largeur de l'écran
from IPython.core.display import HTML
HTML("<style>.container { width:95% }</style>")

# affichage des objets sklearn
set_config(display='diagram')

## Fonctions en cours de développement (à déplacer ultérieurement dans un fichier .py externe)

## Chargement du pickle du df nettoyé

In [83]:
df1 = pd.read_pickle('df1_après_nettoyage_Projet_Seattle.pkl')

# Suppression des lignes pour lesquelles la variable 'SiteEUI(kBtu/sf)' n'est pas renseignée
df1 = df1.dropna(subset=['SiteEUI(kBtu/sf)']).copy()

# Troisième approche du problème : utilisation des surfaces des PropertyUseType, en regroupant les types rares

Exemple de modalité problématique dans la deuxième approche : 'Swimming Pool' est présent dans plusieurs types de propriétés, mais les GFA sont toutes de 0 pour cette modalité de propriété.

## Rechargement du pickle du df nettoyé

In [84]:
df2 = pd.read_pickle('df1_après_nettoyage_Projet_Seattle.pkl')

## Nombre de propriétés renseignées pour chaque type de surface

Liste qui contient les modalités :

In [85]:
liste_modalites_types_1_2_3 = list(df2['LargestPropertyUseType'].dropna().unique()) + list(
    df2['SecondLargestPropertyUseType'].dropna().unique()) + list(
        df2['ThirdLargestPropertyUseType'].dropna().unique())

In [86]:
liste_modalites_types_1_2_3 = sorted(list(set(liste_modalites_types_1_2_3)))   # suppression doublons
liste_modalites_types_1_2_3

['Adult Education',
 'Automobile Dealership',
 'Bank Branch',
 'Bar/Nightclub',
 'College/University',
 'Convenience Store without Gas Station',
 'Convention Center',
 'Courthouse',
 'Data Center',
 'Distribution Center',
 'Enclosed Mall',
 'Fast Food Restaurant',
 'Financial Office',
 'Fire Station',
 'Fitness Center/Health Club/Gym',
 'Food Sales',
 'Food Service',
 'Hospital (General Medical & Surgical)',
 'Hotel',
 'K-12 School',
 'Laboratory',
 'Library',
 'Lifestyle Center',
 'Manufacturing/Industrial Plant',
 'Medical Office',
 'Movie Theater',
 'Multifamily Housing',
 'Museum',
 'Non-Refrigerated Warehouse',
 'Office',
 'Other',
 'Other - Education',
 'Other - Entertainment/Public Assembly',
 'Other - Lodging/Residential',
 'Other - Mall',
 'Other - Public Services',
 'Other - Recreation',
 'Other - Restaurant/Bar',
 'Other - Services',
 'Other - Technology/Science',
 'Other - Utility',
 'Other/Specialty Hospital',
 'Parking',
 'Performing Arts',
 'Personal Services (Health/Bea

Dictionnaire qui contient le nb de propriétés pour un type de surface donnée :

In [87]:
# initialisation
dict_modalites_types_1_2_3 = {x: 0 for x in liste_modalites_types_1_2_3}

In [88]:
dict_modalites_types_1_2_3

{'Adult Education': 0,
 'Automobile Dealership': 0,
 'Bank Branch': 0,
 'Bar/Nightclub': 0,
 'College/University': 0,
 'Convenience Store without Gas Station': 0,
 'Convention Center': 0,
 'Courthouse': 0,
 'Data Center': 0,
 'Distribution Center': 0,
 'Enclosed Mall': 0,
 'Fast Food Restaurant': 0,
 'Financial Office': 0,
 'Fire Station': 0,
 'Fitness Center/Health Club/Gym': 0,
 'Food Sales': 0,
 'Food Service': 0,
 'Hospital (General Medical & Surgical)': 0,
 'Hotel': 0,
 'K-12 School': 0,
 'Laboratory': 0,
 'Library': 0,
 'Lifestyle Center': 0,
 'Manufacturing/Industrial Plant': 0,
 'Medical Office': 0,
 'Movie Theater': 0,
 'Multifamily Housing': 0,
 'Museum': 0,
 'Non-Refrigerated Warehouse': 0,
 'Office': 0,
 'Other': 0,
 'Other - Education': 0,
 'Other - Entertainment/Public Assembly': 0,
 'Other - Lodging/Residential': 0,
 'Other - Mall': 0,
 'Other - Public Services': 0,
 'Other - Recreation': 0,
 'Other - Restaurant/Bar': 0,
 'Other - Services': 0,
 'Other - Technology/Scien

In [89]:
def compte_type_propriete(categorie):
    for indice, modalite, surface in zip(df2[categorie].index,   # supprimer itérateur indice
                                         df2[categorie],
                                         df2[categorie+'GFA']):
        if ~np.isnan(surface):
            if surface != 0:
                dict_modalites_types_1_2_3[modalite] +=1
    
compte_type_propriete('LargestPropertyUseType')
compte_type_propriete('SecondLargestPropertyUseType')
compte_type_propriete('ThirdLargestPropertyUseType')


In [90]:
# Tri du résultat
{k: v for k, v in sorted(dict_modalites_types_1_2_3.items(), key=lambda item: item[1])}

{'Swimming Pool': 0,
 'Convention Center': 1,
 'Enclosed Mall': 1,
 'Fast Food Restaurant': 1,
 'Fire Station': 1,
 'Police Station': 1,
 'Residential Care Facility': 1,
 'Wholesale Club/Supercenter': 1,
 'Courthouse': 2,
 'Food Sales': 2,
 'Lifestyle Center': 2,
 'Other - Technology/Science': 2,
 'Vocational School': 2,
 'Convenience Store without Gas Station': 3,
 'Other - Utility': 3,
 'Prison/Incarceration': 3,
 'Adult Education': 4,
 'Bar/Nightclub': 4,
 'Library': 4,
 'Movie Theater': 4,
 'Other - Mall': 4,
 'Personal Services (Health/Beauty, Dry Cleaning, etc)': 4,
 'Urgent Care/Clinic/Other Outpatient': 4,
 'Museum': 5,
 'Other - Public Services': 5,
 'Other/Specialty Hospital': 5,
 'Performing Arts': 5,
 'Other - Lodging/Residential': 6,
 'Automobile Dealership': 7,
 'Strip Mall': 7,
 'Other - Education': 8,
 'Financial Office': 9,
 'Other - Restaurant/Bar': 9,
 'Food Service': 10,
 'Repair Services (Vehicle, Shoe, Locksmith, etc)': 10,
 'Hospital (General Medical & Surgical)'

## Feature engineering

Pour chaque modalité de PropertyUseType, on ajoute une colonne au dataframe df2.  
Cette colonne contiendra la superficie pour la modalité considérée.  
Cependant, les types de propriété avec moins de N occurences seront regroupés dans la variable 'Rare'.  

In [91]:
N = 2

##### Ajout de colonne à df2 pour chaque PropertyUseType avec >N occurences

In [92]:
# Création d'une liste contenant le nom des colonnes créés
liste_variables_PropertyUseTypes_GFA = []

for moda in liste_modalites_types_1_2_3:
    if dict_modalites_types_1_2_3[moda] > N :
        nom_moda_GFA = moda + ' GFA'
        df2[nom_moda_GFA] = 0   # initialisation à 0 sq/f
        liste_variables_PropertyUseTypes_GFA.append(nom_moda_GFA)
    
# Colonne 'Rare GFA'
liste_variables_PropertyUseTypes_GFA.append('Rare GFA')
df2['Rare GFA'] = 0

In [93]:
df2

,OSEBuildingID,BuildingType,PrimaryPropertyType,ZipCode,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),TotalGHGEmissions,GHGEmissionsIntensity,Adult Education GFA,Automobile Dealership GFA,Bank Branch GFA,Bar/Nightclub GFA,College/University GFA,Convenience Store without Gas Station GFA,Data Center GFA,Distribution Center GFA,Financial Office GFA,Fitness Center/Health Club/Gym GFA,Food Service GFA,Hospital (General Medical & Surgical) GFA,Hotel GFA,K-12 School GFA,Laboratory GFA,Library GFA,Manufacturing/Industrial Plant GFA,Medical Office GFA,Movie Theater GFA,Multifamily Housing GFA,Museum GFA,Non-Refrigerated Warehouse GFA,Office GFA,Other GFA,Other - Education GFA,Other - Entertainment/Public Assembly GFA,Other - Lodging/Residential GFA,Other - Mall GFA,Other - Public Services GFA,Other - Recreation GFA,Other - Restaurant/Bar GFA,Other - Services GFA,Other - Utility GFA,Other/Specialty Hospital GFA,Parking GFA,Performing Arts GFA,"Personal Services (Health/Beauty, Dry Cleaning, etc) GFA",Pre-school/Daycare GFA,Prison/Incarceration GFA,Refrigerated Warehouse GFA,"Repair Services (Vehicle, Shoe, Locksmith, etc) GFA",Residence Hall/Dormitory GFA,Restaurant GFA,Retail Store GFA,Self-Storage Facility GFA,Senior Care Community GFA,Social/Meeting Hall GFA,Strip Mall GFA,Supermarket/Grocery Store GFA,Urgent Care/Clinic/Other Outpatient GFA,Worship Facility GFA,Rare GFA
0,1,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61220,-122.33799,1927,1.0,12,88434,0,88434,Hotel,88434.0,NaN,NaN,NaN,NaN,60.0,81.699997,84.300003,182.500000,189.000000,7.226362e+06,7.456910e+06,2003882.00,1.156514e+06,3.946027e+06,12764.529300,1.276453e+06,249.98,2.83,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61317,-122.33393,1996,1.0,11,103566,15064,88502,Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,61.0,94.800003,97.900002,176.100006,179.399994,8.387933e+06,8.664479e+06,0.00,9.504252e+05,3.242851e+06,51450.816410,5.145082e+06,295.86,2.86,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61393,-122.33810,1969,1.0,41,956110,196718,759392,Hotel,756493.0,NaN,NaN,NaN,NaN,43.0,96.000000,97.699997,241.899994,244.100006,7.258702e+07,7.393711e+07,21566554.00,1.451544e+07,4.952666e+07,14938.000000,1.493800e+06,2089.28,2.19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61412,-122.33664,1926,1.0,10,61320,0,61320,Hotel,61320.0,NaN,NaN,NaN,NaN,56.0,110.800003,113.300003,216.199997,224.000000,6.794584e+06,6.946800e+06,2214446.25,8.115253e+05,2.768924e+06,18112.130860,1.811213e+06,286.43,4.67,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,NonResidential,Hotel,98121.0,7,DOWNTOWN,47.61375,-122.34047,1980,1.0,18,175580,62000,113580,Hotel,123445.0,Parking,68009.0,Swimming Pool,NaN,75.0,114.800003,118.699997,211.399994,215.600006,1.417261e+07,1.465650e+07,0.00,1.573449e+06,5.368607e+06,88039.984380,8.803998e+06,505.01,2.88,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [94]:
liste_variables_PropertyUseTypes_GFA

['Adult Education GFA',
 'Automobile Dealership GFA',
 'Bank Branch GFA',
 'Bar/Nightclub GFA',
 'College/University GFA',
 'Convenience Store without Gas Station GFA',
 'Data Center GFA',
 'Distribution Center GFA',
 'Financial Office GFA',
 'Fitness Center/Health Club/Gym GFA',
 'Food Service GFA',
 'Hospital (General Medical & Surgical) GFA',
 'Hotel GFA',
 'K-12 School GFA',
 'Laboratory GFA',
 'Library GFA',
 'Manufacturing/Industrial Plant GFA',
 'Medical Office GFA',
 'Movie Theater GFA',
 'Multifamily Housing GFA',
 'Museum GFA',
 'Non-Refrigerated Warehouse GFA',
 'Office GFA',
 'Other GFA',
 'Other - Education GFA',
 'Other - Entertainment/Public Assembly GFA',
 'Other - Lodging/Residential GFA',
 'Other - Mall GFA',
 'Other - Public Services GFA',
 'Other - Recreation GFA',
 'Other - Restaurant/Bar GFA',
 'Other - Services GFA',
 'Other - Utility GFA',
 'Other/Specialty Hospital GFA',
 'Parking GFA',
 'Performing Arts GFA',
 'Personal Services (Health/Beauty, Dry Cleaning, e

##### Ajout des surfaces des PropertyUseType dans le dataframe df2

In [95]:
def ajout_surfaces_2(categorie):
    for indice, modalite, surface in zip(df2[categorie].index,
                                         df2[categorie],
                                         df2[categorie+'GFA']):
        if ~np.isnan(surface):
            if modalite + ' GFA' in liste_variables_PropertyUseTypes_GFA:
                df2.at[indice, modalite + ' GFA'] += surface
            else:
                df2.at[indice, 'Rare GFA'] += surface
    
ajout_surfaces_2('LargestPropertyUseType')
ajout_surfaces_2('SecondLargestPropertyUseType')
ajout_surfaces_2('ThirdLargestPropertyUseType')

In [96]:
df2.sample(100, random_state=0)

,OSEBuildingID,BuildingType,PrimaryPropertyType,ZipCode,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),TotalGHGEmissions,GHGEmissionsIntensity,Adult Education GFA,Automobile Dealership GFA,Bank Branch GFA,Bar/Nightclub GFA,College/University GFA,Convenience Store without Gas Station GFA,Data Center GFA,Distribution Center GFA,Financial Office GFA,Fitness Center/Health Club/Gym GFA,Food Service GFA,Hospital (General Medical & Surgical) GFA,Hotel GFA,K-12 School GFA,Laboratory GFA,Library GFA,Manufacturing/Industrial Plant GFA,Medical Office GFA,Movie Theater GFA,Multifamily Housing GFA,Museum GFA,Non-Refrigerated Warehouse GFA,Office GFA,Other GFA,Other - Education GFA,Other - Entertainment/Public Assembly GFA,Other - Lodging/Residential GFA,Other - Mall GFA,Other - Public Services GFA,Other - Recreation GFA,Other - Restaurant/Bar GFA,Other - Services GFA,Other - Utility GFA,Other/Specialty Hospital GFA,Parking GFA,Performing Arts GFA,"Personal Services (Health/Beauty, Dry Cleaning, etc) GFA",Pre-school/Daycare GFA,Prison/Incarceration GFA,Refrigerated Warehouse GFA,"Repair Services (Vehicle, Shoe, Locksmith, etc) GFA",Residence Hall/Dormitory GFA,Restaurant GFA,Retail Store GFA,Self-Storage Facility GFA,Senior Care Community GFA,Social/Meeting Hall GFA,Strip Mall GFA,Supermarket/Grocery Store GFA,Urgent Care/Clinic/Other Outpatient GFA,Worship Facility GFA,Rare GFA
2738,26761,NonResidential,Small- and Mid-Sized Office,98105.0,4,LAKE UNION,47.653730,-122.327210,2007,1.0,3,44510,18228,26282,Office,26282.0,Parking,18228.0,NaN,NaN,23.0,83.300003,85.599998,261.399994,268.899994,2.188051e+06,2.250450e+06,0.000000e+00,6.412810e+05,2188051.00,0.000000,0.000000e+00,15.25,0.34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26282,0,0,0,0,0,0,0,0,0,0,0,18228,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1839,23562,NonResidential,Mixed Use Property,98104.0,7,DOWNTOWN,47.601950,-122.335160,1913,1.0,3,43878,0,43878,Multifamily Housing,12000.0,Office,8262.0,Retail Store,6312.0,NaN,23.400000,23.400000,73.500000,73.500000,7.697713e+05,7.697713e+05,0.000000e+00,2.256070e+05,769771.00,0.000000,0.000000e+00,5.37,0.12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12000,0,0,8262,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6312,0,0,0,0,0,0,0,0
1139,21088,NonResidential,Small- and Mid-Sized Office,98103.0,4,LAKE UNION,47.649040,-122.346090,1985,1.0,3,31417,8110,23307,Office,26295.0,Parking,12890.0,NaN,NaN,94.0,42.799999,46.200001,99.599998,102.099998,1.126157e+06,1.214409e+06,0.000000e+00,2.014866e+05,687472.00,4386.850098,4.386850e+05,28.09,0.89,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26295,0,0,0,0,0,0,0,0,0,0,0,12890,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1043,20613,NonResidential,Warehouse,98134.0,2,GREATER DUWAMISH,47.569820,-122.334790,1951,1.0,1,24000,0,24000,Non-Refrigerated Warehouse,20400.0,Office,3600.0,NaN,NaN,NaN,52.400002,62.900002,93.000000,104.099998,1.257034e+06,1.510110e+06,0.000000e+00,1.278410e+05,436193.00,8208.400391,8.208400e+05,46.64,1.94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20400,3600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2076,24374,NonResidential,Retail Store,98118.0,2,SOUTHEAST,47.565140,-122.287810,1968,1.0,1,36071,0,36071,Retail Store,36071.0,NaN,NaN,NaN,NaN,97.0,42.099998,42.099998,81.900002,81.900002,1.519952e+06,1.519952e+06,0.000000e+00,1.903151e+05,649355.00,8705.969727,8.705970e+05,50.76,1.41,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36071,0,0,0,0,0,0,0,0
921,20237,NonResidential,Retail Stor

## Train test split 

POUR LE NOTEBOOK FINAL IL FAUDRA FAIRE LE SPLIT PLUS TOT (avant le nettoyage)  
IL FAUDRA FAIRE LE MEME NETTOYAGE SUR LES JEUX TRAIN ET TEST

POUR L'INSTANT ON NE TRAVAILLE QU'A LA PREDICTION DE SiteEUI(kBtu/sf)

In [97]:
df2 = df2.dropna(subset=['SiteEnergyUse(kBtu)']).copy()

##### Variable à prédire :

In [98]:
y4 = df2[['SiteEnergyUse(kBtu)']].copy()

##### Variables explicatives :

In [99]:
X4 = df2[liste_variables_PropertyUseTypes_GFA].copy()

##### Split :

In [100]:
X_train_EU, X_test_EU, y_train_EU, y_test_EU = train_test_split(X4, y4, test_size=0.2, random_state=0)

## Régression linéaire simple, sans rescaling, cross-validation

In [101]:
lin_reg = LinearRegression();

In [102]:
lin_scores = cross_val_score(lin_reg, X_train_EU, y_train_EU,
                             scoring="neg_root_mean_squared_error", cv=10)
display_scores(-lin_scores, puissance10=6)

Scores: 5.55e6 8.68e6 9.99e6 5.76e6 14.83e6 6.24e6 16.08e6 6.86e6 22.67e6 9.79e6 
Mean: 10.65e6
Standard deviation: 5.58e6


In [103]:
lin_scores = cross_val_score(lin_reg, X_train_EU, y_train_EU,
                             scoring="r2", cv=10)
display_scores(lin_scores)

Scores: 0.842 0.262 0.863 0.566 0.756 0.691 0.955 0.57 0.675 0.582 
Mean: 0.676
Standard deviation: 0.197


On passe de RMSE=10.7e6 (baseline / N=-1) à 10.7e6 (N=10).  
On passe de R²=0.678 (baseline / N=-1) à :
- 0.650 (N=20),  
- 0.640 (N=15),  
- 0.640 (N=12),  
- 0.675 (N=10),  
- 0.672 (N=8),  
- 0.670 (N=5),  
- 0.676 (N=2),  
- 0.678 (N=1).
- 0.678 (N=0).

## Données recalibrées par StandardScaler

In [104]:
ss = StandardScaler()
X_train_EU_ss = ss.fit_transform(X_train_EU)
X_test_EU_ss = ss.transform(X_test_EU)

### Régression Ridge

In [105]:
ridge_ss = RidgeCV(alphas=np.logspace(-2, 2, 100), cv=10)    # 100 valeurs alpha entre 1e0 et 1e2
ridge_ss.fit(X_train_EU_ss, y_train_EU)

RidgeCV(alphas=array([1.00000000e-02, 1.09749877e-02, 1.20450354e-02, 1.32194115e-02, 1.45082878e-02, 1.59228279e-02, 1.74752840e-02, 1.91791026e-02, 2.10490414e-02,
       2.31012970e-02, 2.53536449e-02, 2.78255940e-02, 3.05385551e-02, 3.35160265e-02, 3.67837977e-02, 4.03701726e-02, 4.43062146e-02, 4.86260158e-02,
       5.33669923e-02, 5.85702082e-02, 6.42807312e-02, 7.05480231e-0...
       8.11130831e+00, 8.90215085e+00, 9.77009957e+00, 1.07226722e+01, 1.17681195e+01, 1.29154967e+01, 1.41747416e+01, 1.55567614e+01, 1.70735265e+01,
       1.87381742e+01, 2.05651231e+01, 2.25701972e+01, 2.47707636e+01, 2.71858824e+01, 2.98364724e+01, 3.27454916e+01, 3.59381366e+01, 3.94420606e+01,
       4.32876128e+01, 4.75081016e+01, 5.21400829e+01, 5.72236766e+01, 6.28029144e+01, 6.89261210e+01, 7.56463328e+01, 8.30217568e+01, 9.11162756e+01,
       1.00000000e+02]),
        cv=10)

In [106]:
best_alpha = ridge_ss.alpha_
best_alpha

9.770099572992256

In [107]:
# R²
R2_ss_ridge_cv = arrondi(ridge_ss.best_score_)
R2_ss_ridge_cv

0.678

On passe de 0.680 (baseline / N=-1) à :
- 0.650 (N=20).  
- 0.640 (N=15).  
- 0.676 (N=10).  
- 0.640 (N=12). 
- 0.673 (N=8).  
- 0.671 (N=5).
- 0.678 (N=2).  
- 0.680 (N=1).  
- 0.680 (N=0).  

##### R² sur le test set avec le meilleur alpha issu de la CV

In [108]:
ridge_ss = Ridge(alpha=best_alpha)
ridge_ss.fit(X_train_EU_ss, y_train_EU)
print(arrondi(ridge_ss.score(X_test_EU_ss, y_test_EU)))

0.836


On passe de 0.815 (baseline / N=-1) à :
- 0.773 (N=20).  
- 0.801 (N=15). 
- 0.801 (N=12).
- 0.84  (N=10).
- 0.838 (N=8).  
- 0.838 (N=5).  
- 0.836 (N=2).  
- 0.816 (N=1).  
- 0.815 (N=0).  

##### Conclusion sur la 3è approche :

L'utilisation des surfaces des PropertyUseType, en regroupant les types rares implique un effet défavorable sur le train set et favorable sur le test set …